In [3]:
import add_packages
import yaml, requests, re
from crewai import Agent, Task, Crew, Process
from pprint import pprint
from my_crewai import tools as my_tools
from my_langchain import chat_models

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
	configs_crewai = yaml.safe_load(f)

configs = configs_crewai

params = {
	"origin": "Ho Chi Minh City, Viet Nam",
	"travel_dates": "November 2024",
	"destination": "Da Lat, Viet Nam",
	"interests": "Hiking, Yoga, Sightseeing",
}

# Base

In [ ]:
import re

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
  configs_crewai = yaml.safe_load(f)

configs = configs_crewai
configs_agents_traveling = configs_crewai["agents"]["traveling"]
configs_tasks_traveling = configs_crewai["tasks"]["traveling"]

params = {
	"origin": "Atlanta, GA",
	"travel_dates": "November 2024",
	"destination": "Thialand",
	"interests": "Hiking, Yoga, Sightseeing",
}

def _substitute_placeholders_in_configs(configs, params):
	pattern = r'%(\w+)%'

	def repl(match):
		placeholder = match.group(1)
		if placeholder.lower() in params:
			return str(params[placeholder.lower()])
		return match.group(0)

	return {
		key: {k: re.sub(pattern, repl, v) for k, v in value.items()}
		for key, value in configs.items()
	}


# Substitute placeholders in agent and task configurations
configs_agents_traveling_substituted = _substitute_placeholders_in_configs(
	configs_agents_traveling, params)
configs_tasks_traveling_substituted = _substitute_placeholders_in_configs(
	configs_tasks_traveling, params)

# Create agents
manager = Agent(
	**configs_agents_traveling_substituted["manager"],
	verbose=True,
	allow_delegation=True,
	llm=chat_models.chat_openai,
)

travel_agent = Agent(
	**configs_agents_traveling_substituted["travel_consultant"],
	tools=[
		my_tools.ToolsCalculator().calculate
	],
	max_iter=7,
	llm=chat_models.chat_openai,
)

city_selection_expert = Agent(
	**configs_agents_traveling_substituted["city_selection_expert"],
	verbose=True,
	tools=[
		my_tools.ToolsBrowser().search_duckduckgo,
		my_tools.ToolsContent().read_content
	],
	llm=chat_models.chat_openai,
)

local_tour_guide = Agent(
	**configs_agents_traveling_substituted["local_tour_guide"],
	verbose=True,
	tools=[
		my_tools.ToolsBrowser().search_duckduckgo,
		my_tools.ToolsContent().read_content
	],
	llm=chat_models.chat_openai,
)

quality_control_expert = Agent(
	**configs_agents_traveling_substituted["quality_control_expert"],
	verbose=True,
	tools=[] + my_tools.tools_human,
	llm=chat_models.chat_openai,
)

# Create tasks
manager_task = Task(
	**configs_tasks_traveling_substituted["manage"],
	agent=manager,
	async_execution=False,
)

identify_city = Task(
	**configs_tasks_traveling_substituted["identify_city"],
	agent=city_selection_expert,
	async_execution=False
)

gather_city_info = Task(
	**configs_tasks_traveling_substituted["gather_city_info"],
	agent=city_selection_expert,
	async_execution=False
)

plan_itinerary = Task(
	**configs_tasks_traveling_substituted["plan_itinerary"],
	agent=travel_agent,
	async_execution=False,
)

quality_control = Task(
	**configs_tasks_traveling_substituted["quality_control"],
	agent=quality_control_expert,
	context=[plan_itinerary],
	async_execution=False
)

# Forming the crew with a hierarchical process including the manager
crew = Crew(
	agents=[
		manager,
		travel_agent,
		city_selection_expert,
		local_tour_guide,
		quality_control_expert],
	tasks=[manager_task,
		   plan_itinerary,
		   identify_city,
		   gather_city_info,
		   quality_control],
	process=Process.hierarchical,
	manager_llm=chat_models.chat_openai,
	verbose=2,
)

In [ ]:

# Kick off the crew's work
results = crew.kickoff()

# Print the results
print("Crew Work Results:")
print(results)

# final

In [ ]:
class CustomCrew:
	def __init__(self, params, configs, task_name) -> None:
		self.params = params
		self.configs_agents = configs["agents"][task_name]
		self.configs_tasks = configs["tasks"][task_name]
		self.agents = {}
		self._load_agents()
		self.tasks = self._create_tasks()
		self.crew = self._create_crew()

	def _substitute_placeholders_in_configs(self, configs):
		pattern = r'%(\w+)%'

		def repl(match):
			placeholder = match.group(1)
			if placeholder.lower() in self.params:
				return str(self.params[placeholder.lower()])
			return match.group(0)
		return {
			key: {k: re.sub(pattern, repl, v) for k, v in value.items()}
			for key, value in configs.items()
		}

	def _load_agents(self):
		configs_agents_substituted = self._substitute_placeholders_in_configs(
				self.configs_agents)

		for agent_key, agent_config in configs_agents_substituted.items():
			agent_tools = self._get_agent_tools(agent_key)

			agent_config["allow_delegation"] = agent_config.get(
				"allow_delegation", "false").lower() == "true"

			self.agents[agent_key] = Agent(
				**agent_config,
				tools=agent_tools,
				verbose=True,
				llm=chat_models.chat_openai, 
			)

	def _get_agent_tools(self, agent_key):
		# Logic to assign tools based on agent key
		if agent_key == "manager":
			return [
				my_tools.ToolsCalculator().calculate
			]
		elif agent_key == "travel_consultant":
			return [
				my_tools.ToolsCalculator().calculate,
				my_tools.ToolsBrowser().search_duckduckgo,
				my_tools.ToolsContent().read_content
			]
		elif agent_key == "city_selection_expert":
			return [
				my_tools.ToolsBrowser().search_duckduckgo,
				my_tools.ToolsContent().read_content
			]
		elif agent_key == "local_tour_guide":
			return [
				my_tools.ToolsBrowser().search_duckduckgo,
				my_tools.ToolsContent().read_content
			]
		elif agent_key == "quality_control_expert":
			return [] + my_tools.tools_human
		else:
			return []

	def _create_tasks(self):
			configs_tasks_substituted = self._substitute_placeholders_in_configs(
							self.configs_tasks)
			tasks = {}

			for task_key, task_config in configs_tasks_substituted.items():
					task_config["async_execution"] = task_config.get(
									"async_execution", "false").lower() == "true"

					# Initialize agents for this task
					agent_name = task_config.get("agent", "")
					task_agent = self.agents[agent_name]
					
					task_config["agent"] = task_agent
     
					tasks[task_key] = Task(
						**task_config,
					)
			return tasks

	def _create_crew(self):
		return Crew(
			agents=list(self.agents.values()),
			tasks=list(self.tasks.values()),
			process=Process.hierarchical,
			manager_llm=chat_models.chat_openai,
			verbose=2
		)

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
	configs_crewai = yaml.safe_load(f)

configs = configs_crewai

params = {
	"origin": "Ho Chi Minh City, Viet Nam",
	"travel_dates": "November 2024",
	"destination": "Da Lat, Viet Nam",
	"interests": "Hiking, Yoga, Sightseeing",
}
task_name = "traveling"

custom_crew = CustomCrew(params, configs, task_name)

In [ ]:
custom_crew.crew.kickoff()

In [ ]:
dict(custom_crew.agents["quality_control_expert"])

# test

In [4]:
class CustomCrew:
	def __init__(self, params, configs, task_name, tool_dict) -> None:
		self.params = params
		self.configs_agents = configs["agents"][task_name]
		self.configs_tasks = configs["tasks"][task_name]
		self.agents = {}
		self.tools = tool_dict
		self._load_agents()
		self.tasks = self._create_tasks()
		self.crew = self._create_crew()


	def _substitute_placeholders_in_configs(self, configs):
			if not configs:
					return {}
			
			pattern = r'%(\w+)%'
			def repl(match):
					placeholder = match.group(1)
					if placeholder.lower() in self.params:
							return str(self.params[placeholder.lower()])
					return match.group(0)
			
			return {
					key: {k: re.sub(pattern, repl, v) for k, v in value.items()}
					for key, value in configs.items()
			}

	def _load_agents(self):
		configs_agents_substituted = self._substitute_placeholders_in_configs(
				self.configs_agents)

		for agent_key, agent_config in configs_agents_substituted.items():
				tools_str = agent_config.get("tools")
				if tools_str == "x":
						agent_tools = []
				elif tools_str:
						agent_tools = [
								self.tools[tool_name.strip()] for tool_name in tools_str.split(",")
						]
				else:
						agent_tools = []

				agent_config["allow_delegation"] = agent_config.get(
						"allow_delegation", "false").lower() == "true"
				agent_config["tools"] = agent_tools

				self.agents[agent_key] = Agent(
						**agent_config,
						verbose=True,
						llm=chat_models.chat_openai,
				)
    
	def _create_tasks(self):
			configs_tasks_substituted = self._substitute_placeholders_in_configs(
							self.configs_tasks)
			tasks = {}

			for task_key, task_config in configs_tasks_substituted.items():
					task_config["async_execution"] = task_config.get(
									"async_execution", "false").lower() == "true"

					# Initialize agents for this task
					agent_name = task_config.get("agent", "")
					task_agent = self.agents[agent_name]
					
					task_config["agent"] = task_agent
     
					tasks[task_key] = Task(
						**task_config,
					)
			return tasks

	def _create_crew(self):
		return Crew(
			agents=list(self.agents.values()),
			tasks=list(self.tasks.values()),
			process=Process.hierarchical,
			manager_llm=chat_models.chat_openai,
			verbose=2
		)

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
	configs_crewai = yaml.safe_load(f)

configs = configs_crewai

params = {
	"origin": "Ho Chi Minh City, Viet Nam",
	"travel_dates": "November 2024",
	"destination": "Da Lat, Viet Nam",
	"interests": "Hiking, Yoga, Sightseeing",
}

task_name = "traveling"

tools_crew = {
	"calculator": my_tools.ToolsCalculator().calculate,
	"searcher": my_tools.ToolsBrowser().search_duckduckgo,
	"reader": my_tools.ToolsContent().read_content,
	"human": my_tools.tools_human,
}

custom_crew = CustomCrew(params, configs, task_name, tools_crew)


In [5]:
dict(custom_crew.agents["city_selection_expert"])["tools"]

[DuckDuckGoSearchRun(),
 StructuredTool(name='Read webpage content', description='Read webpage content(url: str) -> str - Read content from a webpage.', args_schema=<class 'pydantic.v1.main.Read webpage contentSchema'>, func=<function ToolsContent.read_content at 0x168743380>)]

In [5]:
custom_crew.crew.kickoff()

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Oversee the integration of research findings, trip suggestions, and quality control feedback to produce a 7-day travel itinerary specifically tailored to the user's interests: Hiking, Yoga, Sightseeing and their travel destination: Da Lat, Viet Nam. The final output should be a comprehensive with detailed per-day plans, including budget, packing suggestions.



> Entering new CrewAgentExecutor chain...
I need to ensure that the team integrates research findings, trip suggestions, and quality control feedback effectively to create a tailored 7-day travel itinerary for the user's interests in Hiking, Yoga, and Sightseeing in Da Lat, Viet Nam. I must oversee the process and make sure all details are included in the final output.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Travel Manager",
    "task": "Research the best hiking trails in Da Lat, Viet Nam",
    "context": "Provide detailed information on t

'# 7-Day Travel Itinerary to Da Lat, Viet Nam\n\n## Trip Date: November 2024\n## Traveling from: Ho Chi Minh City, Viet Nam\n## Traveling to: Da Lat, Viet Nam\n## Traveler Interests: Hiking, Yoga, Sightseeing\n\n### Day 1: Arrival in Da Lat\n- **Morning:** Arrive in Da Lat via Vietnam Airlines flight from Ho Chi Minh City.\n- **Afternoon:** Check into the centrally located Dalat Palace Heritage Hotel, known for its French Colonial architecture and luxurious amenities.\n- **Evening:** Enjoy a welcome dinner at the Art Cafe, offering a blend of Vietnamese and French cuisine.\n\n### Day 2: Langbiang Mountain Trail Hike\n- **Morning:** Start your day with a hearty breakfast at the hotel.\n- **Afternoon:** Embark on a guided hike along the Langbiang Mountain Trail, offering breathtaking views of the city and surrounding landscapes.\n- **Evening:** Relax at the hotel spa and unwind with a yoga session.\n\n### Day 3: Cultural Exploration\n- **Morning:** Visit the Linh Phuoc Pagoda, known for 